In [25]:
import pandas as pd
import numpy as np

import yaml

from catboost import CatBoostRegressor, Pool, metrics
from sklearn.model_selection import train_test_split

In [26]:
# config params
def config():
    with open("../config/params.yaml", "r") as f:
            return yaml.safe_load(f)

cfg = config()

# data split cfg
test_size = cfg['data_split']['test_size']
test_split_seed = cfg['data_split']['test_split_seed']
valid_size = cfg['data_split']['valid_size']
valid_split_seed = cfg['data_split']['valid_split_seed']

# tuning params
test_size = cfg['data_split']['test_size']

In [27]:
df = pd.read_csv('https://github.com/EuMentality/datasets/raw/main/taxi_train.csv', sep=';')

In [28]:
df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_month,pickup_weekday,pickup_hour,high_traffic,anomaly,manh_length
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,6.122493,3,0,17,1,0,2.006876
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,6.498282,6,6,0,0,0,2.394278
2,2,1,-73.979027,40.763939,-74.005333,40.710087,N,7.661527,1,1,11,1,0,7.304991
3,2,1,-74.010040,40.719971,-74.012268,40.706718,N,6.063785,4,2,19,1,0,1.930130
4,2,1,-73.973053,40.793209,-73.972923,40.782520,N,6.077642,3,5,13,1,0,1.608787


In [63]:
cat_feature_indices = np.array([0, 1, 6, 8, 9, 10, 11])
X = df.drop('trip_duration', axis=1)
X.iloc[:, cat_feature_indices] = X.iloc[:, cat_feature_indices].astype(str)
y = df.trip_duration

In [64]:
cfg = config()
cfg_split = cfg['data_split']
cfg_hprms = cfg['hyperparams']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=20)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=20)

In [72]:
model = CatBoostRegressor(
            loss_function='RMSE',
            random_seed=20,
            early_stopping_rounds=25,
            verbose=100
            )

In [75]:
model.fit(X_train, y_train,
         cat_features=cat_feature_indices,
         eval_set=(X_valid, y_valid))

Learning rate set to 0.142665
0:	learn: 0.6663251	test: 0.6682139	best: 0.6682139 (0)	total: 148ms	remaining: 2m 27s
100:	learn: 0.4226705	test: 0.4258532	best: 0.4258532 (100)	total: 12.6s	remaining: 1m 51s
200:	learn: 0.4132376	test: 0.4173542	best: 0.4173542 (200)	total: 26s	remaining: 1m 43s
300:	learn: 0.4086339	test: 0.4136738	best: 0.4136738 (300)	total: 38.5s	remaining: 1m 29s
400:	learn: 0.4052051	test: 0.4111904	best: 0.4111904 (400)	total: 50.5s	remaining: 1m 15s
500:	learn: 0.4028380	test: 0.4096838	best: 0.4096838 (500)	total: 1m 3s	remaining: 1m 3s
600:	learn: 0.4007728	test: 0.4085810	best: 0.4085810 (600)	total: 1m 17s	remaining: 51.6s
700:	learn: 0.3988359	test: 0.4075460	best: 0.4075460 (700)	total: 1m 32s	remaining: 39.4s
800:	learn: 0.3971837	test: 0.4069013	best: 0.4068966 (799)	total: 1m 46s	remaining: 26.5s
900:	learn: 0.3955470	test: 0.4062224	best: 0.4062216 (899)	total: 1m 59s	remaining: 13.2s
999:	learn: 0.3942625	test: 0.4056558	best: 0.4056558 (999)	total: 

In [79]:
model.save_model('../model/catboost_test.dump')

In [80]:
# asd = CatBoostRegressor()
# asd.load_model('../model/catboost_test.dump')